## Finilizing Data Generation for 0D Reactor

In [1]:
### Importing Libraries

import sys
print(sys.version)
import os
import numpy as np
import pandas as pd
import time

import matplotlib.pyplot as plt
WORKSPACE_PATH  = os.environ['WORKSPACE_PATH']
plt.style.use(WORKSPACE_PATH+'/ROMNet/romnet/extra/postprocessing/presentation.mplstyle')

from scipy.integrate import solve_ivp

3.8.11 (default, Jun 29 2021, 03:17:05) 
[Clang 12.0.0 (clang-1200.0.32.29)]


In [2]:
### Input Data

OutputDir  = WORKSPACE_PATH+'/ROMNet/Data/0DReact_Isobaric_1Cond/'
try:
    os.makedirs(OutputDir)
except OSError as e:
    pass

tStratch           = 1.
SOLVER             = 'BDF'

ValidPerc          = 20.

FixedMinVal        = 1.e-14

In [3]:
FileName = OutputDir+'/Orig/train/ext/CleanVars.csv'
Vars     = pd.read_csv(FileName, delimiter=',', header=None).to_numpy()[0,:]

In [4]:

Vars0 = []
for Var in Vars:
    Vars0.append(str(Var)+'0')
    

FileName     = OutputDir + '/Orig/train/ext/SimIdxs.csv'
Data         = pd.read_csv(FileName, header=0)
SimIdxs      = Data.to_numpy(int)


FileName     = OutputDir + '/Orig/train/ext/ICs.csv'
Data         = pd.read_csv(FileName)
ICVecs       = Data.to_numpy()
NICs         = len(ICVecs)


P0Vec        = []
EqRatio0Vec  = []
T0Vec        = []
for iC in range(NICs):
    Nts      = SimIdxs[iC,1]-SimIdxs[iC,0]
    P0Vec.append(       np.ones(Nts, dtype=np.float64)*ICVecs[iC,0])
    EqRatio0Vec.append( np.ones(Nts, dtype=np.float64)*ICVecs[iC,1])
    T0Vec.append(       np.ones(Nts, dtype=np.float64)*ICVecs[iC,2])
P0Vec        = np.concatenate(P0Vec)[...,np.newaxis]
EqRatio0Vec  = np.concatenate(EqRatio0Vec)[...,np.newaxis]
T0Vec        = np.concatenate(T0Vec)[...,np.newaxis]


# for iC in range(NICs):
#     Nts              = SimIdxs[iC,1]-SimIdxs[iC,0]
#     FileName         = OutputDir+'/'+str(NVarsRed)+'PC/train/ext/PC.csv.'+str(iC+1)
#     DataTemp         = pd.read_csv(FileName, header=0)
#     DataTemp         = DataTemp[DataTemp['t'] == 0.][Vars]
#     DataTemp         = DataTemp.loc[DataTemp.index.repeat(Nts)]
#     if (iC==0):
#         Data0        = DataTemp
#     else:
#         Data0        = pd.concat([Data0, DataTemp], axis=0)
# Data0.columns = Vars0
# Data0.reset_index(drop=True, inplace=True)
for iC in range(NICs):
    FileName         = OutputDir+'/Orig/train/ext/y.csv.'+str(iC+1)
    DataNew          = pd.read_csv(FileName, header=0)
    Nts              = len(DataNew)
    DataTemp         = DataNew.iloc[[0]][Vars]#DataTemp.iloc[1][Vars]
    DataTemp         = DataTemp.loc[DataTemp.index.repeat(Nts)]
    DataTemp.reset_index(drop=True, inplace=True)
    DataNew[Vars0]   = DataTemp
    DataTemp['t']    = DataNew['t']
    DataNew['t']     = DataNew['t'].to_numpy() - DataTemp['t'].iloc[[0]].to_numpy()
    if (iC==0):
        Data        = DataNew
    else:
        Data        = pd.concat([Data, DataNew], axis=0)
Data.reset_index(drop=True, inplace=True)

In [5]:
Data

,t,T,H2,H,O,O2,OH,H2O,HO2,H2O2,...,H2O20,N0,NH0,NH30,NNH0,NO0,NO20,N2O0,HNO0,N20
0,0.000000e+00,999.999930,0.013800,0.000000e+00,0.000000e+00,0.219060,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.767141
1,3.691671e-11,999.999930,0.013800,2.914757e-17,1.959065e-17,0.219060,1.260459e-17,1.850670e-17,1.089269e-15,7.627498e-20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.767141
2,7.396970e-11,999.999930,0.013800,5.840274e-17,3.925361e-17,0.219060,2.525571e-17,3.708173e-17,2.182558e-15,1.528315e-19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.767141
3,1.111595e-10,999.999930,0.013800,8.776591e-17,5.898917e-17,0.219060,3.795353e-17,5.572533e-17,3.279884e-15,2.296707e-19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.767141
4,1.484865e-10,999.999930,0.013800,1.172375e-16,7.879759e-17,0.219060,5.069823e-17,7.443775e-17,4.381261e-15,3.067936e-19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.767141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,9.853686e-01,2160.761062,0.000041,3.975943e-06,3.715529e-04,0.104082,2.907047e-03,1.213995e-01,4.425541e-06,2.062407e-07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.767138
49996,9.890062e-01,2160.752447,0.000041,3.975656e-06,3.715273e-04,0.104080,2.906926e-03,1.213996e-01,4.425331e-06,2.062333e-07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.767138
49997,9.926573e-01,2160.743897,0.000041,3.975371e-06,3.715019e-04,0.104077,2.906805e-03,1.213997e-01,4.425122e-06,2.062260e-07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.767138
49998,9.963219e-01,2160.735412,0.000041,3.975089e-06,3.714767e-04,0.104075,2.906686e-03,1.213998e-01,4.424915e-06,2.062187e-07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.767138


In [6]:
from sklearn.model_selection import train_test_split

try:
    os.makedirs(OutputDir+'/Orig/train/')
except:
    pass
try:
    os.makedirs(OutputDir+'/Orig/valid/')
except:
    pass

In [7]:
data_id    = 'pts'

DataNoZero           = Data[Data['t'] >= FixedMinVal]
n_points             = len(DataNoZero)

idx                  = np.arange(n_points)
train_idx, valid_idx = train_test_split(idx, test_size=ValidPerc/100, random_state=42)

n_valid              = len(valid_idx)
n_train              = len(train_idx)

try:
    os.makedirs(OutputDir+'/Orig/train/'+data_id+'/')
except:
    pass
try:
    os.makedirs(OutputDir+'/Orig/valid/'+data_id+'/')
except:
    pass

DataInput  = DataNoZero[['t'] + list(Vars0)]
DataInput.iloc[train_idx].to_csv(OutputDir+'/Orig/train/'+data_id+'/Input.csv', index=False)
DataInput.iloc[valid_idx].to_csv(OutputDir+'/Orig/valid/'+data_id+'/Input.csv', index=False)

DataOutput = DataNoZero[['t'] + list(Vars)]
DataOutput.iloc[train_idx].to_csv(OutputDir+'/Orig/train/'+data_id+'/Output.csv', index=False)
DataOutput.iloc[valid_idx].to_csv(OutputDir+'/Orig/valid/'+data_id+'/Output.csv', index=False)